In [12]:
import findspark
findspark.init()

from pyspark.sql import SparkSession
from pyspark.sql.functions import explode, split, lower, count

In [2]:
spark_session = SparkSession.builder.appName('Word Count').getOrCreate()
data = spark_session.read.text('../data/Book')

In [3]:
words_df = data.select(explode(split(data.value, '\\W+')).alias('words'))
words_without_empty_string = words_df.filter(words_df.words != '')

In [10]:
lowercase_words = words_without_empty_string.select(lower(words_without_empty_string.words).alias('lowercase_words'))

In [17]:
word_counts = lowercase_words.groupBy('lowercase_words') \
                             .agg(count(lowercase_words.lowercase_words).alias('frequency')) \
                             .sort('frequency', ascending=False) \
                             .show(5)

+---------------+---------+
|lowercase_words|frequency|
+---------------+---------+
|            you|     1878|
|             to|     1828|
|           your|     1420|
|            the|     1292|
|              a|     1191|
+---------------+---------+
only showing top 5 rows



In [18]:
spark_session.stop()